In [11]:
import pandas as pd
import numpy as np
import pickle
import pyarrow

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [12]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green_taxi_model_new")

<Experiment: artifact_location='gs://mlflow-backend-storage-models/1', creation_time=1728422822438, experiment_id='1', last_update_time=1728422822438, lifecycle_stage='active', name='green_taxi_model_new', tags={}>

In [13]:
import mlflow
import os
from google.oauth2 import service_account

# Set up credentials with proper scopes
credentials = service_account.Credentials.from_service_account_file(
    '/home/MERHAWI/arrival_time/Arrival_time_estimation/mlflow_cld/my_gcp_k.json',
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

# Configure MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/MERHAWI/arrival_time/Arrival_time_estimation/mlflow_cld/my_gcp_k.json'

In [14]:
def read_dataframe(filename:str):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration > 1) & (df.duration <=60)]

    categorical = ['PULocationID','DOLocationID']
    df[categorical]= df[categorical].astype(str)
    return df

def prepare_dictionaries(df: pd.DataFrame):    
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [15]:
df_train = read_dataframe('../data/green_tripdata_2023-01.parquet')
df_valid = read_dataframe('../data/green_tripdata_2023-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_valid = df_valid[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_valid)

In [16]:
# from test_gcp_access import setup_mlflow_with_gcs, log_artifact_example

# # Setup MLflow
# experiment_id = setup_mlflow_with_gcs()

# # Log your artifact
# log_artifact_example("test.txt", "my_artifacts")

# import mlflow

# mlflow.set_tracking_uri("http://127.0.0.1:5000")
# mlflow.set_experiment("green_taxi_model_new")

# with mlflow.start_run():
#     # Just provide the local file path and optional subfolder
#     mlflow.log_artifact("test.txt", "my_folder")  # Don't include gs:// here

In [17]:
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
# mlflow.set_experiment("green_taxi_model_new")
# mlflow.log_artifact("test.txt", artifact_path="gs://mlflow-backend-storage-models")


In [19]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs= -1)
    
    X_train = dv.fit_transform(dict_train)
    model.fit(X_train, y_train)
    
    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

    rmse = root_mean_squared_error(y_pred, y_valid)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(model, artifact_path="model")

    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('dict_vectorizer.bin')
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 5.396698280020155


2024/10/08 21:46:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/08 21:46:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run nebulous-gull-820 at: http://127.0.0.1:5000/#/experiments/1/runs/ba222bda45b94132b337915e4fa8226d.
2024/10/08 21:46:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.
